In [20]:
import json
import requests
import pandas as pd

In [2]:
with open("../../secrets.json", 'r') as fr:
    content = "".join(fr.readlines())
    API_KEY = json.loads(content)['eod_histo_data_api_key']

### Get Exchanges

In [3]:
params = {'api_token': API_KEY}
exchanges = requests.get(url="https://eodhistoricaldata.com/api/exchanges-list/", params=params)
len(json.loads(exchanges.text))

80

In [41]:
json.loads(exchanges.text)

[{'Name': 'USA Stocks',
  'Code': 'US',
  'OperatingMIC': 'XNAS, XNYS',
  'Country': 'USA',
  'Currency': 'USD'},
 {'Name': 'London Exchange',
  'Code': 'LSE',
  'OperatingMIC': 'XLON',
  'Country': 'UK',
  'Currency': 'GBP'},
 {'Name': 'Toronto Exchange',
  'Code': 'TO',
  'OperatingMIC': 'XTSE',
  'Country': 'Canada',
  'Currency': 'CAD'},
 {'Name': 'TSX Venture Exchange',
  'Code': 'V',
  'OperatingMIC': 'XTSX',
  'Country': 'Canada',
  'Currency': 'CAD'},
 {'Name': 'NEO Exchange',
  'Code': 'NEO',
  'OperatingMIC': 'NEOE',
  'Country': 'Canada',
  'Currency': 'CAD'},
 {'Name': 'Hamburg Exchange',
  'Code': 'HM',
  'OperatingMIC': 'XHAM',
  'Country': 'Germany',
  'Currency': 'EUR'},
 {'Name': 'Munich Exchange',
  'Code': 'MU',
  'OperatingMIC': 'XMUN',
  'Country': 'Germany',
  'Currency': 'EUR'},
 {'Name': 'XETRA Exchange',
  'Code': 'XETRA',
  'OperatingMIC': 'XETR',
  'Country': 'Germany',
  'Currency': 'EUR'},
 {'Name': 'Dusseldorf Exchange',
  'Code': 'DU',
  'OperatingMIC': '

### Get Symbols

In [105]:
EXCHANGE_CODE = 'LSE'
url=f"https://eodhistoricaldata.com/api/exchange-symbol-list/{EXCHANGE_CODE}?api_token={API_KEY}"
tickers = requests.get(url=url)
len(tickers.text.split("\n"))

6310

In [49]:
tickers.text.split("\n")

['Code,Name,Country,Exchange,Currency,Type,Isin',
 '0A00,"Akzo Nobel N.V.",UK,LSE,EUR,"Common Stock",',
 '0A02,"Neoen S.A.",UK,LSE,EUR,"Common Stock",FR0011675362',
 '0A05,"Medacta Group S.A.",UK,LSE,CHF,"Common Stock",',
 '0A0C,"Stadler Rail AG",UK,LSE,CHF,"Common Stock",CH0002178181',
 '0A0D,"Alcon Inc.",UK,LSE,USD,"Common Stock",',
 '0A0F,"Citycon Oyj",UK,LSE,EUR,"Common Stock",FI4000369947',
 '0A0H,"Beijer Ref AB Series B",UK,LSE,SEK,"Common Stock",',
 '0A0I,"Wihlborgs Fastigheter AB (publ)",UK,LSE,SEK,"Common Stock",',
 '0A0J,"AAK AB",UK,LSE,SEK,"Common Stock",',
 '0A0K,"Nyfosa AB",UK,LSE,SEK,"Common Stock",',
 '0A1K,"NIO Inc. ADR",UK,LSE,USD,"Common Stock",US62914V1061',
 '0A1V,"Tethys Oil AB",UK,LSE,SEK,"Common Stock",SE0015949847',
 '0A28,"Prosus N.V.",UK,LSE,EUR,"Common Stock",',
 '0A29,"Solutions 30 SE",UK,LSE,EUR,"Common Stock",',
 '0A2N,"Lynas Rare Earths Ltd.",UK,LSE,AUD,"Common Stock",',
 '0A37,"Betsson AB Series B",UK,LSE,SEK,"Common Stock",SE0015672282',
 '0A39,"Karnov 

In [50]:
type_counts = {}
for elem in tickers.text.split("\n"):
    sub_elems = elem.split(",")
    if len(sub_elems) == 7:
        type_ = sub_elems[5]
        if type_ in type_counts.keys():
            type_counts[type_] += 1
        else:
            type_counts[type_] = 1
type_counts


{'Type': 1,
 '"Common Stock"': 4108,
 'FUND': 5,
 'ETF': 1688,
 'INDEX': 1,
 'ETC': 24}

In [51]:
exchange_counts = {}
for elem in tickers.text.split("\n"):
    sub_elems = elem.split(",")
    if len(sub_elems) == 7:
        type_ = sub_elems[3]
        if type_ in exchange_counts.keys():
            exchange_counts[type_] += 1
        else:
            exchange_counts[type_] = 1
exchange_counts

{'Exchange': 1, 'LSE': 5826}

### Get Historical Quotes

In [41]:
FROM = "2022-02-23"
TO = "2022-03-23"

In [42]:
TICKER = "NVDA.US"

In [110]:
url = f"https://eodhistoricaldata.com/api/eod/{TICKER}?api_token={API_KEY}&from={FROM}&to={TO}"
quotes = requests.get(url=url)

In [111]:
# &fmt=json   pd.read_json(quotes.text)
df = pd.read_csv(url, skipfooter=1)

/var/folders/qt/k5txb7556tz0pk9dmksrt2sr0000gn/T/ipykernel_57205/2928175860.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(url, skipfooter=1)


In [112]:
df.head()

,Date,Open,High,Low,Close,Adjusted_close,Volume
0,2022-02-23,382.72,386.00,366.66,367.46,367.46,4614300
1,2022-02-24,355.09,390.73,354.72,390.03,390.03,7068700
2,2022-02-25,386.61,391.29,375.58,390.80,390.80,4841600
3,2022-02-28,387.33,397.75,382.13,394.52,394.52,5030600
4,2022-03-01,391.60,395.00,383.71,386.24,386.24,3290400


#### Split/Dividend Adjustment

In [100]:
df['k'] = df['Close'] / df['Adjusted_close']
df['Open'] = df['Open'] / df['k']
df['High'] = df['High'] / df['k']
df['Low'] = df['Low'] / df['k']
df.drop(['Close', 'k'], axis='columns', inplace=True)
df.rename(columns={"Adjusted_close": "Close"}, inplace=True)

In [106]:
df.head()

,Date,Open,High,Low,Close,Volume
0,2022-02-23,237.929202,241.457756,222.924928,223.7846,56651078
1,2022-02-24,210.069827,237.909202,208.820303,237.3894,73580062
2,2022-02-25,237.119464,242.077571,233.720762,241.4778,52886551
3,2022-02-28,239.828400,246.555831,236.979589,243.7569,47731859
4,2022-03-01,242.817293,243.676965,231.231717,234.6804,41205602


### Get Dividends And Splits

In [88]:
url = f"https://eodhistoricaldata.com/api/div/{TICKER}?api_token={API_KEY}&from={FROM}&fmt=json"
dividends = requests.get(url=url)

In [89]:
dividends.json()

[{'date': '2017-05-19',
  'declarationDate': None,
  'recordDate': '2017-05-23',
  'paymentDate': '2017-06-14',
  'period': 'Quarterly',
  'value': 0.035,
  'unadjustedValue': 0.14,
  'currency': 'USD'},
 {'date': '2017-08-22',
  'declarationDate': '2017-08-10',
  'recordDate': '2017-08-24',
  'paymentDate': '2017-09-18',
  'period': 'Quarterly',
  'value': 0.035,
  'unadjustedValue': 0.14,
  'currency': 'USD'},
 {'date': '2017-11-22',
  'declarationDate': '2017-11-09',
  'recordDate': '2017-11-24',
  'paymentDate': '2017-12-15',
  'period': 'Quarterly',
  'value': 0.0375,
  'unadjustedValue': 0.15,
  'currency': 'USD'},
 {'date': '2018-02-22',
  'declarationDate': '2018-02-08',
  'recordDate': '2018-02-23',
  'paymentDate': '2018-03-16',
  'period': 'Quarterly',
  'value': 0.0375,
  'unadjustedValue': 0.15,
  'currency': 'USD'},
 {'date': '2018-05-23',
  'declarationDate': '2018-05-10',
  'recordDate': '2018-05-24',
  'paymentDate': '2018-06-15',
  'period': 'Quarterly',
  'value': 0.

In [90]:
url = f"https://eodhistoricaldata.com/api/splits/{TICKER}?api_token={API_KEY}&from={FROM}"
splits = requests.get(url=url)

In [91]:
splits.text

'Date,"Stock Splits"\n2021-07-20,4.000000/1.000000\n49'

### Speedtest historical quotes

In [104]:
for TICKER in ["NVDA.US", "TSLA.US", "MSFT.US", "AMZN.US", "NFLX.US"]:
    url = f"https://eodhistoricaldata.com/api/eod/{TICKER}?api_token={API_KEY}&from={FROM}&to={TO}"
    quotes = requests.get(url=url)
    # print("-", end="")
    # url = f"https://eodhistoricaldata.com/api/div/{TICKER}?api_token={API_KEY}&from={FROM}&fmt=json"
    # dividends = requests.get(url=url)
    # print("-", end="")
    # url = f"https://eodhistoricaldata.com/api/splits/{TICKER}?api_token={API_KEY}&from={FROM}"
    # splits = requests.get(url=url)
    print("-")


-
-
-
-
-
